loading libraries

In [1]:
import numpy as np
import json as js
import tensorflow as tf
from keras import models
from keras.utils import register_keras_serializable #type: ignore
import cv2
from os import listdir as ld
from os import path
from tqdm import tqdm

2025-04-21 15:50:30.806225: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-21 15:50:30.818216: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-21 15:50:30.821666: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-21 15:50:30.830601: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Loading paths

In [2]:
with open("config_CNN.json",'r') as file:
    paths = js.load(file)

loading model

In [3]:
def compute_iou_tf(box1, box2):
    box1_x1 = box1[:, 0]
    box1_y1 = box1[:, 1]
    box1_x2 = box1[:, 0] + box1[:, 2]
    box1_y2 = box1[:, 1] + box1[:, 3]
    box2_x1 = box2[:, 0]
    box2_y1 = box2[:, 1]
    box2_x2 = box2[:, 0] + box2[:, 2]
    box2_y2 = box2[:, 1] + box2[:, 3]
    x1 = tf.maximum(box1_x1, box2_x1)
    y1 = tf.maximum(box1_y1, box2_y1)
    x2 = tf.minimum(box1_x2, box2_x2)
    y2 = tf.minimum(box1_y2, box2_y2)
    intersection = tf.maximum(0.0, x2 - x1) * tf.maximum(0.0, y2 - y1)
    area1 = box1[:, 2] * box1[:, 3]
    area2 = box2[:, 2] * box2[:, 3]
    union = area1 + area2 - intersection
    return tf.math.divide_no_nan(intersection, union)

In [4]:
@register_keras_serializable()
class IoUMetric(tf.keras.metrics.Metric):
    def __init__(self, name='iou_metric', **kwargs):
        super(IoUMetric, self).__init__(name=name, **kwargs)
        self.total_iou = self.add_weight(name='total_iou', initializer='zeros')
        self.count = self.add_weight(name='count', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.reshape(y_true, (-1, 4))
        y_pred = tf.reshape(y_pred, (-1, 4))

        ious = compute_iou_tf(y_true, y_pred)
        self.total_iou.assign_add(tf.reduce_sum(ious))
        self.count.assign_add(tf.cast(tf.size(ious), tf.float32))

    def result(self):
        return tf.math.divide_no_nan(self.total_iou, self.count)

    def reset_states(self):
        self.total_iou.assign(0.0)
        self.count.assign(0.0)


In [5]:
model = models.load_model(paths["Trained_model"], custom_objects={"IoUMetric": IoUMetric})

I0000 00:00:1745230832.875184  651773 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1745230832.916142  651773 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1745230832.918181  651773 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1745230832.923760  651773 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

Evaluation on validation data

In [6]:
def compute_iou(box1, box2):
    x1_min = box1[0]
    y1_min = box1[1]
    x1_max = box1[0] + box1[2]
    y1_max = box1[1] + box1[3]
    x2_min = box2[0]
    y2_min = box2[1]
    x2_max = box2[0] + box2[2]
    y2_max = box2[1] + box2[3]
    inter_xmin = max(x1_min, x2_min)
    inter_ymin = max(y1_min, y2_min)
    inter_xmax = min(x1_max, x2_max)
    inter_ymax = min(y1_max, y2_max)
    inter_area = max(0, inter_xmax - inter_xmin) * max(0, inter_ymax - inter_ymin)
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)
    union_area = box1_area + box2_area - inter_area
    return inter_area / union_area if union_area > 0 else 0

TP, FP, FN = 0, 0, 0


In [7]:
val_data_path = paths["Validation_resized"]
iou_threshold = 0.001

with open(paths["Preprocessed_Validation_json"], 'r') as file:
    val_json = js.load(file)

gt_boxes_dict = {item["img_id"]: item["bbox"] for item in val_json}
TP = 0
FP = 0
FN = 0

for fname in tqdm(ld(val_data_path), desc="Evaluating"):
    if not fname.endswith((".jpg", ".png", ".jpeg")):
        continue

    if fname not in gt_boxes_dict:
        continue

    img_path = path.join(val_data_path, fname)
    img = cv2.imread(img_path)

    if img is None:
        print(f"Warning: Failed to read {fname}")
        continue

    img_input = np.expand_dims(img, axis=0)
    pred = model.predict(img_input, verbose=0)
    pred_box = pred['bbox_output'][0]
    gt_box = gt_boxes_dict[fname]
    pred_box = list(map(float, pred_box))
    gt_box = list(map(float, gt_box))
    iou = compute_iou(pred_box, gt_box)

    if iou >= iou_threshold:
        TP += 1
    else:
        FN += 1  


precision = TP / (TP + FP + 1e-6)
recall = TP / (TP + FN + 1e-6)
f1_score = 2 * precision * recall / (precision + recall + 1e-6)

print(f"\n[Evaluation Results]")
print(f"True Positives : {TP}")
print(f"False Positives: {FP}")
print(f"False Negatives: {FN}")
print(f"Precision      : {precision:.4f}")
print(f"Recall         : {recall:.4f}")
print(f"F1 Score       : {f1_score:.4f}")

Evaluating:   0%|          | 0/37000 [00:00<?, ?it/s]I0000 00:00:1745230834.777563  651852 service.cc:146] XLA service 0x7888cc019ac0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745230834.777595  651852 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2025-04-21 15:50:34.790646: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-04-21 15:50:34.918675: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 90101
I0000 00:00:1745230836.915427  651852 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
Evaluating:   6%|▌         | 2120/37000 [01:34<25:50, 22.50it/s]


KeyboardInterrupt: 